In [1]:
import pandas as pd
import json
from datetime import datetime, timedelta

In [2]:
raw_data = pd.read_csv('/Users/ksysoev/Downloads/DE Assessment/data.csv', dtype={'id': 'int64', 'application_date': 'string', 'contracts': 'string'})

In [3]:
raw_data.head()

,id,application_date,contracts
0,2925210,2024-02-12 19:22:46.652000+00:00,<NA>
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
2,2925212,2024-02-12 19:24:41.493000+00:00,<NA>
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa..."
4,2925214,2024-02-12 19:24:56.857000+00:00,<NA>


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                1000 non-null   int64 
 1   application_date  1000 non-null   string
 2   contracts         495 non-null    string
dtypes: int64(1), string(2)
memory usage: 23.6 KB


In [5]:
def parse_contracts(contracts):
    data = [['contract']]
    
    if not pd.isnull(contracts):
        c_df = pd.json_normalize(json.loads(contracts))
        
        try:
            c_df.loc[c_df['summa'] == '', 'summa'] = 0
            c_df['summa'] = c_df['summa'].astype(int)

            c_df.loc[c_df['loan_summa'] == '', 'loan_summa'] = 0
            c_df['loan_summa'] = c_df['loan_summa'].astype(int)

            c_df['contract_date'] = c_df['contract_date'].astype(str)
            c_df['claim_date'] = c_df['claim_date'].astype(str)
            try:
                c_df['bank'] = c_df['bank'].astype(str)
            except KeyError:
                c_df['bank'] = ''
        except ValueError: 
            pass
    else:
        contract = json.loads('{}')
        c_df = pd.DataFrame(data)
        c_df = c_df[1:]
        c_df.reset_index(drop=True)
        
    return c_df

In [6]:
def tot_claim_cnt_l180d(contracts):
    data = parse_contracts(contracts)
    data['filter'] = 0
    for index, row in data.iterrows():
        # print(index)
        try:
            if datetime.strptime(str(row['claim_date']),"%d.%m.%y") > datetime.now() - timedelta(days = 180):
                data.iloc[index]['filter'] = 1
        except ValueError:
            pass
        except KeyError:
            print('KeyError')
            # print(index)
            pass
    
    result = 0
    try:
        result = data[data['filter'] == 1].nunique()['claim_id']
    except KeyError:
        pass
    
    if result == 0:
        return -3
    
    return result

In [7]:
def disb_bank_loan_wo_tbc(contracts):
    data = parse_contracts(contracts)
    list1 = ['LIZ', 'LOM', 'MKO', 'SUG']
    try:
        data = data[(~data['bank'].isin(list1)) & (~data['bank'].isna()) & (data['contract_date'] != '')]
    except KeyError:
        return -3
    
    claims_cnt = data.nunique()['claim_id']
    contract_cnt = data.nunique()['contract_id']
    
    if claims_cnt == 0:
        return -3
    elif contract_cnt == 0:
        return -1
    
    return data['loan_summa'].sum()

In [8]:
def day_sinlastloan(application_date, contracts):    
    data = parse_contracts(contracts)

    try:
        data = data[data['summa'] != 0]
        try:
            data['contract_date'].apply(lambda x: datetime.strptime(str(x), "%d.%m.%y"))
        except ValueError:
            pass
        data['last_contract_date'] = data['contract_date'].max()
        try:
            data['days_cnt'] = (datetime.strptime(str(data['last_contract_date'][0]),"%d.%m.%y") - datetime.fromisoformat(str(application_date)).date()).days
        except ValueError:
            pass
    except KeyError:
        return -3
    
    claims_cnt = data.nunique()['claim_id']
    contract_cnt = data.nunique()['contract_id']
    
    if claims_cnt == 0:
        return -3
    elif contract_cnt == 0:
        return -1
    
    return data['loan_summa'].sum()

In [9]:
raw_data['tot_claim_cnt_l180d'] = raw_data['contracts'].apply(lambda x: tot_claim_cnt_l180d(x))

In [10]:
raw_data['disb_bank_loan_wo_tbc'] = raw_data['contracts'].apply(lambda x: disb_bank_loan_wo_tbc(x))

In [11]:
raw_data['day_sinlastloan'] = raw_data.apply(lambda x: day_sinlastloan(x.application_date, x.contracts), axis=1)

In [12]:
raw_data.head()

,id,application_date,contracts,tot_claim_cnt_l180d,disb_bank_loan_wo_tbc,day_sinlastloan
0,2925210,2024-02-12 19:22:46.652000+00:00,<NA>,-3,-3,-3
1,2925211,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",-3,0,0
2,2925212,2024-02-12 19:24:41.493000+00:00,<NA>,-3,-3,-3
3,2925213,2024-02-12 19:24:29.135000+00:00,"[{""contract_id"": 522530, ""bank"": ""003"", ""summa...",-3,0,0
4,2925214,2024-02-12 19:24:56.857000+00:00,<NA>,-3,-3,-3
